In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
from keras.utils.np_utils import to_categorical
import numpy as np
import cv2
import skimage
from skimage import measure
from collections import Counter
from keras.layers import Conv2D,MaxPooling2D,Flatten,Softmax,Activation,Dense,Dropout
from keras import Sequential
from keras import optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from keras.callbacks import Callback,ModelCheckpoint
from tqdm import tqdm
from sklearn.model_selection import  StratifiedKFold
import matplotlib.pyplot as plt
from keras import optimizers
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import load_model

NEW BLOCK OF CODE

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Deep Learning Project/UWI Capstone/Data/Cured Data/Stru Data/Covid19")
Class ={"Positive": 1, "Negative": 0, "Negative; Positive (Confirmed later)": 1 }
Type = {1 : "Positive + Negative; Positive (Confirmed later)", 0: "Negative"}
Classes_txt = ["Patient", "SARS-CoV-2 nucleic acids"]
Classes_txt = df[Classes_txt]
#Classes_txt.to_csv(r'/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Classes_txt.csv', header=None, index=None, sep='\t', mode='a')
Patient_Class ={line.strip().split('\t')[0]:line.strip().split('\t')[1] for line in open('/content/drive/MyDrive/Deep Learning Project/UWI Capstone/Classes_txt.csv') if line.strip().split('\t')[1] in Class}
Patient_Type ={patient:Class[c]for patient,c in Patient_Class.items() if c in Class}


NEW LINE BLOCK


In [ ]:
Patient_Images = "/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Patients Images"

def read_img_dirs(folder):
    file_list=[]
    for root,dirs,files in os.walk(folder):
      for file i
n files:
        file_list.append(os.path.join(root,file).replace("\\",'/'))
    return file_list


CT_Address = read_img_dirs(Patient_Images)

In [ ]:

# Code adopted from iCTC

def corp_margin(img2):
    img2=np.asarray(img2)
    (row, col) = img2.shape
    row_top = 0
    raw_down = 0
    col_top = 0
    col_down = 0
    axis1=img2.sum(axis=1)
    axis0=img2.sum(axis=0)
    for r in range(0, row):
        if axis1[r] > 30:
            row_top = r
            break
    for r in range(row - 1, 0, -1):
        if axis1[r] > 30:
            raw_down = r
            break
    for c in range(0, col):
        if axis0[c] > 30:
            col_top = c
            break
    for c in range(col - 1, 0, -1):
        if axis0[c] > 30:
            col_down = c
            break
    a=raw_down+ 1 - row_top-(col_down+ 1-col_top)
    if a>0:
            w=raw_down+ 1-row_top
            col_down=int((col_top+col_down + 1)/2+w/2)
            col_top = col_down-w
            if col_top < 0:
                col_top = 0
                col_down = col_top + w
            elif col_down >= col:
                col_down = col - 1
                col_top = col_down - w
    else:
            w=col_down + 1- col_top
            raw_down = int((row_top + raw_down + 1) / 2 + w/2)
            row_top =  raw_down-w
            if row_top < 0:
                row_top = 0
                raw_down = row_top + w
            elif raw_down >= row:
                raw_down = row - 1
                row_top = raw_down - w
    if row_top==raw_down:
        row_top=0
        raw_down=99
        col_top = 0
        col_down = 99
    new_img = img2[row_top:raw_down + 1, col_top:col_down + 1]
    return new_img

In [ ]:
def read_ct_img_bydir(target_dir):
    img=cv2.imdecode(np.fromfile(target_dir,dtype=np.uint8),cv2.IMREAD_GRAYSCALE)
    img = corp_margin(img)
    img=cv2.resize(img,(200,200))
    return img

In [ ]:
model = load_model('/content/drive/MyDrive/Deep Learning Project/UWI Capstone/Code/Actual Code/Mod Code/CT_SliceNew_model.h5')

In [ ]:
X_CT_Valid= np.array([read_ct_img_bydir(file) for file in CT_Address])[:,:,:,np.newaxis]
y_CT_Valid= model.predict(X_CT_Valid)

66/66 [==============================] - 154s 2s/step


In [ ]:
df_Slice_CT_Predit =pd.DataFrame({'File':CT_Address,'NiCT':y_CT_Valid[:,0],'pCT':y_CT_Valid[:,1],'nCT':y_CT_Valid[:,2]})
df_Slice_CT_Predit.insert(1,'Patient',df_Slice_CT_Predit['File'].str.split('/').apply(lambda x:x[-3]))
df_Slice_CT_Predit.insert(2,'Type',df_Slice_CT_Predit['Patient'].map(Patient_Type))
df_Slice_CT_Predit.to_csv(r'/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/ct_Slice_group_NewPredict.csv',sep='\t',index=None)


new code block

In [ ]:
df_Slice_CT_Predit =pd.read_csv('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/ct_Slice_group_NewPredict.csv', delimiter = "\t")

In [ ]:
df_Slice_CT_Predit

In [ ]:
def top_time_series(df_Slice_CT_Predit,top=10,time_seq=True,del_deficiency=True,invalid_cutoff=0.49):
    df=df_Slice_CT_Predit[df_Slice_CT_Predit['NiCT']<=invalid_cutoff]
    df.sort_values(['Patient','pCT'],ascending=[1,0],inplace=True)
    if time_seq:
        df=df.groupby(['Patient']).head(top).sort_index()
    else:
        df=df.groupby(['Patient']).head(top)
    group_size=df.groupby('Patient').size()
    del_group=group_size[group_size<top]
    if del_deficiency:
        df=df[~df['Patient'].isin(del_group.index)]
    return df

df_top10 = top_time_series(df_Slice_CT_Predit,10)

In [ ]:
df_top10.to_csv(r'/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/NewTop10.csv',sep='\t',index=None)

In [ ]:
df_top10[df_top10["Type"] == 0]

In [ ]:
def X_y_patient_fromdf(df_top):
    X=[]
    y=[]
    patient_list=[]
    for patient,ds in tqdm(df_top.groupby('Patient')):
        X_patient=np.array([read_ct_img_bydir(file) for file in ds['File'].tolist()])
        X_patient=X_patient[:,:,:,np.newaxis].transpose(3,1,2,0)
        X.append(X_patient)
        y.append(ds['Type'].iloc[0])
        patient_list.append(patient)
    return patient_list,np.concatenate(X), y

patient_list, X, y = X_y_patient_fromdf(df_top10)

100%|██████████| 10/10 [00:00<00:00, 12.26it/s]


In [ ]:
Skf = StratifiedKFold(n_splits=4,shuffle=True)

In [ ]:
for train, val in Skf.split(X, y):
    print("TRAIN:", train, "TEST:", val)

In [ ]:
X_train=X[train]
X_val=X[val]
y_train=to_categorical(np.array(y)[train])
y_val=to_categorical(np.array(y)[val])

In [ ]:
y_val

In [ ]:
def VGG_Simple():
    model=Sequential()
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=(200,200,10),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',kernel_initializer='uniform',activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(16,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(16,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2,activation='sigmoid'))
    model.compile(optimizer=tf.optimizers.Adam(lr=0.0007,decay=0.05),loss='binary_crossentropy',metrics=['accuracy'])
    return model

model = VGG_Simple()
model.summary()

In [ ]:
history= model.fit(X_train, y_train, epochs=300, batch_size=64, class_weight = None, validation_data=(X_val, y_val), verbose=1)

Epoch 1/300
1/1 [==============================] - 4s 4s/step - loss: 0.7018 - accuracy: 0.3750 - val_loss: 0.7357 - val_accuracy: 0.0000e+00
Epoch 2/300
1/1 [==============================] - 3s 3s/step - loss: 0.6549 - accuracy: 0.7500 - val_loss: 0.6608 - val_accuracy: 0.5000
Epoch 3/300
1/1 [==============================] - 3s 3s/step - loss: 1.7417 - accuracy: 0.2500 - val_loss: 0.6473 - val_accuracy: 0.5000
Epoch 4/300
1/1 [==============================] - 3s 3s/step - loss: 0.6347 - accuracy: 0.7500 - val_loss: 0.6830 - val_accuracy: 0.5000
Epoch 5/300
1/1 [==============================] - 3s 3s/step - loss: 0.7092 - accuracy: 0.5000 - val_loss: 0.6945 - val_accuracy: 0.5000
Epoch 6/300
1/1 [==============================] - 3s 3s/step - loss: 0.6719 - accuracy: 0.6250 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 7/300
1/1 [==============================] - 3s 3s/step - loss: 0.6692 - accuracy: 0.7500 - val_loss: 0.6916 - val_accuracy: 0.5000
Epoch 8/300
1/1 [=============

In [ ]:
model.save('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/CT_class_model.h5')
np.save('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/CT_class_history.npy',history.history)

In [ ]:
model2 = load_model('/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/CT_class_model.h5')

In [ ]:
X_CT= X
y_CT= model2.predict(X_CT)

In [ ]:
df_CT_Predit =pd.DataFrame({'Patient':patient_list,'0':y_CT[:,0],'1':y_CT[:,1]})
df_CT_Predit.to_csv(r'/content/drive/MyDrive/Deep Learning Project/Pilot Capstone/Created Models/CT Slice Prediction/ct_class_Predict.csv',sep='\t',index=None)